<h1><center>Automatic Quiz Generation from Documents/Web Page</center></h1>
<h2><center>AI-powered Educational Assistant / Content Generator</center></h2>
<h3><center>Build AI Apps - Beginner Level</center></h3>
<h4><center>Education<center></h4>

## Before you start

In order to complete the project you will need to create a developer account with OpenAI and store your API key as an environment variable. Instructions for these steps are outlined below.

### Create a developer account with OpenAI

1. Go to the [API signup page](https://platform.openai.com/signup). 

2. Create your account (you'll need to provide your email address and your phone number).

3. Go to the [API keys page](https://platform.openai.com/account/api-keys). 

4. Create a new secret key.


5. **Take a copy of it**. (If you lose it, delete the key and create a new one.)

### Add a payment method

OpenAI sometimes provides free credits for the API, but this can vary based on geography. You may need to add debit/credit card details. 

**Using the `gpt-3.5-turbo` model in this project should incur a cost less than 1 US cent (but if you rerun tasks, you will be charged every time).** For more information on pricing, see [OpenAI's pricing page](https://openai.com/pricing).

1. Go to the [Payment Methods page](https://platform.openai.com/account/billing/payment-methods).

2. Click Add payment method.

3. Fill in your card details.

### Install open ai library

In [15]:
# !pip install openai

### Load Open AI Key

In [16]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
print(find_dotenv())

OPEN_API_KEY=os.getenv('OPENAI_API_KEY')

# print(OPEN_API_KEY)


C:\Users\agaan\openaiapps\.env


### Create Client

In [17]:

from openai import OpenAI

# Define Model
model="gpt-4o-mini"

#define client
client = OpenAI(api_key=OPEN_API_KEY)

## Quiz Generation Application

In [18]:
# response=client.chat.completions.create(model=model,
#                               messages=[{"role":"user","content":"Tell me more about gravity like I am a 5 year old kid in 2 lines"}
#                                        ]
#                               )

# print(response.choices[0].message.content)

### Read Data From Text File

In [19]:
text="""Earth is the third planet from the Sun and the only astronomical object known to harbor life. This is enabled by Earth being an ocean world, the only one in the Solar System sustaining liquid surface water"""

In [23]:
# Safely read a limited number of lines from a text file
def read_text_from_file(filename, max_lines=100):
    try:
        with open(filename, 'r') as file:
            return ''.join([line for _, line in zip(range(max_lines), file)])
    except FileNotFoundError:
        print(f"Error: {filename} not found.")
        return ""

# Read content from the file
content = read_text_from_file("physics_lecture.txt")

# print(content)

### Read Data From Web Page

In [31]:
import requests
from bs4 import BeautifulSoup

def read_text_from_webpage(url, max_paragraphs=15):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise error if request failed

        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract text from paragraphs in the main content area
        paragraphs = soup.find_all('p')
        selected_text = "\n".join(p.get_text() for p in paragraphs[:max_paragraphs])
        
        return selected_text

    except Exception as e:
        print(f"Error reading from {url}: {e}")
        return ""
content = read_text_from_webpage("https://en.wikipedia.org/wiki/Earth")

### Define Prompts

In [20]:
system_prompt = """
You are a teaching assistant that generates multiple-choice questions from a provided educational text.
Your role is to assist educators by creating quiz questions with one correct answer.
"""

user_prompt="""
Generate a multiple-choice quiz question from the given text:
Format:
Question: <Generated Question>
Options:
a) <Option 1>

b) <Option 2>

c) <Option 3>

d) <Option 4>

Answer: <Correct Option>
"""

### Function To Generate Question

In [32]:

# def generate_quiz_questions(text):
#     quiz_data_list=[]
    
#     for i in range(5):
#         response=client.chat.completions.create(model=model,
#                                       messages=[
#                                           {"role":"system","content":system_prompt},
#                                           {"role":"user","content":user_prompt + content}
#                                                ],
#                                     max_tokens=500
#                                       )
    
#         question_and_answer=response.choices[0].message.content
#         quiz_data_list.append(question_and_answer)
#     return quiz_data_list

# # # Generate quiz questions from the content provided
# quiz_data = generate_quiz_questions(content)

# # # View the first question and answer set
# quiz_data[0]

"Question: What percentage of Earth's crust is covered by the global ocean?\nOptions:\na) 29.2%\n\nb) 40.0%\n\nc) 70.8%\n\nd) 58.5%\n\nAnswer: c) 70.8%"

In [33]:
# quiz_data[4]

"Question: What is the main component of Earth's atmosphere that helps maintain surface conditions by acting as a greenhouse gas?\n\nOptions:\na) Carbon dioxide (CO2)\n\nb) Water vapor\n\nc) Nitrogen\n\nd) Oxygen\n\nAnswer: b) Water vapor"

In [34]:
def split_text_into_chunks(text, max_tokens=300):
    # Naive split based on sentences;
    import textwrap
    sentences = text.split('. ')
    chunks = []
    current_chunk = ''
    for sentence in sentences:
        if len(current_chunk.split()) + len(sentence.split()) < max_tokens:
            current_chunk += sentence + '. '
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + '. '
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks[:5]  # Limit to top 5 chunks for now


In [35]:

def generate_quiz_questions(text):
    quiz_data_list=[]
    chunks=split_text_into_chunks(content)
    for chunk in chunks:
        response=client.chat.completions.create(model=model,
                                      messages=[
                                          {"role":"system","content":system_prompt},
                                          {"role":"user","content":user_prompt + content}
                                               ],
                                    max_tokens=500
                                      )
    
        question_and_answer=response.choices[0].message.content
        quiz_data_list.append(question_and_answer)
    return quiz_data_list

# # Generate quiz questions from the content provided
quiz_data = generate_quiz_questions(content)

# # View the first question and answer set
quiz_data[0]

"Question: Which layer of Earth is responsible for generating its magnetosphere?  \nOptions:  \na) Earth's crust  \nb) Liquid outer core  \nc) Solid inner core  \nd) Atmosphere  \n\nAnswer: b) Liquid outer core"

In [36]:
quiz_data[3]

"Question: What percentage of Earth's crust is covered by its global ocean?\nOptions:\na) 40.8%\n\nb) 50.2%\n\nc) 70.8%\n\nd) 29.2%\n\nAnswer: c) 70.8%"

In [ ]:
# !pip install gradio

In [41]:
# import gradio as gr

# with gr.Blocks() as demo:
#     gr.Markdown("# Hello World App")
#     name=gr.Textbox(label="Enter your name")
#     greet_btn=gr.Button("Greet")
#     output=gr.Textbox()

#     def greet(name):
#         return f"Hello {name}"
#     greet_btn.click(greet,inputs=name,outputs=output)

# demo.launch()

In [44]:
def process_input(source_selection,file,url):
    if file:
        text=read_text_from_file(file)
    elif url:
        text=read_text_from_webpage(url)
    else:
        return "Please upload a file or enter URL"
    quiz_data_list=generate_quiz_questions(text)
    quiz_output="\n\n".join(quiz_data_list)
    return quiz_output
    


In [45]:
import gradio as gr
with gr.Blocks() as demo:
    gr.Markdown("🧪 Quiz Generator from Text File or Webpage")
    input_method=gr.Radio(["Upload File","Enter URL"],label="Select Input Method")
    file_input=gr.File(label="Upload a File(.txt)")
    url_input=gr.Textbox(label="Enter a webpage URL")
    submit_btn=gr.Button("Generate Quiz")
    output=gr.Markdown()

    submit_btn.click(fn=process_input,
                    inputs=[input_method,file_input,url_input],
                    outputs=output)

demo.launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.
